In [2]:
# KOMÓRKA DIAGNOSTYCZNA
import sys
import torch
import torchtext

print("--- DIAGNOSTYKA ŚRODOWISKA ---")
print(f"Ścieżka do interpretera Python: {sys.executable}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja torchtext: {torchtext.__version__}")
print(f"Lokalizacja torchtext: {torchtext.__file__}")
print("---------------------------------")

--- DIAGNOSTYKA ŚRODOWISKA ---
Ścieżka do interpretera Python: c:\Users\lchec\miniconda3\envs\ling-lab\python.exe
Wersja PyTorch: 2.3.0
Wersja torchtext: 0.18.0
Lokalizacja torchtext: c:\Users\lchec\miniconda3\envs\ling-lab\lib\site-packages\torchtext\__init__.py
---------------------------------


In [3]:
# Krok 1: Importy i ustawienia
import os
import torch
import torch.nn as nn
import math
import time
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from speakleash import Speakleash

# Ustawienia
base_dir = "speakleash_data"
dataset_name = "wolne_lektury_corpus"
batch_size = 20
eval_batch_size = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Używane urządzenie: {device}")

c:\Users\lchec\miniconda3\envs\ling-lab\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\lchec\miniconda3\envs\ling-lab\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\lchec\miniconda3\envs\ling-lab\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNI

Używane urządzenie: cpu


In [4]:
# Krok 2: Pobieranie i wczytywanie danych
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

sl = Speakleash(base_dir)

# Krok 2.1: Upewnienie się, że dane są pobrane
# Sprawdzamy, czy dane już istnieją, żeby nie pobierać za każdym razem
if not os.path.exists(os.path.join(base_dir, dataset_name)):
    print(f"Pobieranie zbioru danych: {dataset_name}...")
    sl.get(dataset_name)
    print("Pobieranie zakończone.")
else:
    print(f"Zbiór danych {dataset_name} już istnieje.")

Pobieranie zbioru danych: wolne_lektury_corpus...
Pobieranie zakończone.


In [5]:
# Krok 2.2: Wczytanie danych do pamięci
texts = []
dataset = sl.get(dataset_name)
for doc in dataset.data:
    texts.append(doc)
    if len(texts) >= 1000:
        break

corpus = " ".join(texts)
print(f"Wczytano {len(texts)} dokumentów.")
print(f"Rozmiar korpusu: {len(corpus)} znaków.")

AttributeError: 'NoneType' object has no attribute 'data'

In [ ]:

# Krok 3: Tokenizacja i budowa słownika

tokenizer = get_tokenizer('basic_english')
tokens = tokenizer(corpus)

vocab = build_vocab_from_iterator([tokens], specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab.set_default_index(vocab["<unk>"])

print(f"Liczba tokenów: {len(tokens)}")
print(f"Rozmiar słownika: {len(vocab)}")

NameError: name 'corpus' is not defined

In [ ]:

# Krok 4: Przygotowanie tensorów z danymi

def data_process(raw_text_iter, vocab, tokenizer):
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

train_iter = [corpus] 
data = data_process(train_iter, vocab, tokenizer)

def batchify(data, bsz):
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

# Podział na zbiór treningowy, walidacyjny i testowy (90%, 5%, 5%)
n = data.size(0)
train_data = batchify(data[:int(n*0.9)], batch_size)
val_data = batchify(data[int(n*0.9):int(n*0.95)], eval_batch_size)
test_data = batchify(data[int(n*0.95):], eval_batch_size)

print("Kształt danych treningowych:", train_data.shape)
print("Kształt danych walidacyjnych:", val_data.shape)
print("Kształt danych testowych:", test_data.shape)

bptt = 35
def get_batch(source, i):
    """Pobiera sekwencję wejściową i docelową dla modelu."""
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

Kształt danych treningowych: torch.Size([281476, 20])
Kształt danych walidacyjnych: torch.Size([31275, 10])
Kształt danych testowych: torch.Size([31275, 10])


In [ ]:

# Krok 5: Definicja modelu LSTM

class LSTMModel(nn.Module):
    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.model_type = 'LSTM'
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, hidden):
        emb = self.drop(self.encoder(src))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, len(vocab))
        return decoded, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                weight.new_zeros(self.nlayers, bsz, self.nhid))

In [ ]:

# Krok 6: Definicja modelu Transformer

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = nn.TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output.view(-1, len(vocab))

In [ ]:

# Krok 7: pętla treningowa i ewaluacyjna

def train(model, data_source, optimizer, scheduler, criterion, epoch):
    model.train()
    total_loss = 0.
    start_time = time.time()
    
    if isinstance(model, LSTMModel):
        hidden = model.init_hidden(batch_size)
    
    src_mask = None

    for batch, i in enumerate(range(0, data_source.size(0) - 1, bptt)):
        data, targets = get_batch(data_source, i)
        optimizer.zero_grad()

        if isinstance(model, TransformerModel):
            if src_mask is None or src_mask.size(0) != len(data):
                src_mask = model.generate_square_subsequent_mask(len(data)).to(device)
            output = model(data, src_mask)
        else: 
            hidden = tuple([h.detach() for h in hidden])
            output, hidden = model(data, hidden)
        
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print(f'| epoka {epoch:3d} | {batch:5d}/{len(data_source) // bptt:5d} paczek | '
                  f'lr {scheduler.get_last_lr()[0]:02.2f} | ms/paczkę {elapsed * 1000 / log_interval:5.2f} | '
                  f'strata {cur_loss:5.2f} | ppl {math.exp(cur_loss):8.2f}')
            
            checkpoint = {
                'epoch': epoch,
                'batch': batch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': cur_loss,
            }
            torch.save(checkpoint, f'checkpoint_{model.model_type.lower()}.pt')
            
            total_loss = 0
            start_time = time.time()

def evaluate(model, eval_data, criterion):
    model.eval()
    total_loss = 0.
    
    if isinstance(model, LSTMModel):
        hidden = model.init_hidden(eval_batch_size)
        
    src_mask = None
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            if isinstance(model, TransformerModel):
                if src_mask is None or src_mask.size(0) != len(data):
                    src_mask = model.generate_square_subsequent_mask(len(data)).to(device)
                output = model(data, src_mask)
            else: 
                hidden = tuple([h.detach() for h in hidden])
                output, hidden = model(data, hidden)
            
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:

# Krok 8: Trening modelu LSTM 

ntokens = len(vocab)
emsize = 200
nhid = 200
nlayers = 2
dropout = 0.2
lr = 5.0
epochs = 1 
max_training_time_hours = 4 
max_training_time_seconds = max_training_time_hours * 3600

model_lstm = LSTMModel(ntokens, emsize, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_lstm = torch.optim.SGD(model_lstm.parameters(), lr=lr)
scheduler_lstm = torch.optim.lr_scheduler.StepLR(optimizer_lstm, 1.0, gamma=0.95)

best_val_loss = float('inf')
total_training_start_time = time.time()

print(f"=== Rozpoczynam trening LSTM (limit: {max_training_time_hours}h) ===")
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model_lstm, train_data, optimizer_lstm, scheduler_lstm, criterion, epoch)
    val_loss = evaluate(model_lstm, val_data, criterion)
    
    elapsed_total_time = time.time() - total_training_start_time
    
    print('-' * 89)
    print(f'| koniec epoki {epoch:3d} | czas: {(time.time() - epoch_start_time):5.2f}s | '
          f'strata walidacyjna {val_loss:5.2f} | ppl walidacyjny {math.exp(val_loss):8.2f}')
    print(f'| Całkowity czas treningu: {elapsed_total_time/3600:.2f}h / {max_training_time_hours}h')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_lstm.state_dict(), 'best_model_lstm.pt')
        print("Zapisano nowy najlepszy model LSTM.")

    scheduler_lstm.step()
    
    if elapsed_total_time >= max_training_time_seconds:
        print(f"Przekroczono maksymalny czas treningu. Zakończono.")
        break


=== Rozpoczynam trening LSTM (limit: 4h) ===
| epoka   1 |   200/ 8042 paczek | lr 5.00 | ms/paczkę 3373.74 | strata  9.78 | ppl 17710.72
| epoka   1 |   200/ 8042 paczek | lr 5.00 | ms/paczkę 3373.74 | strata  9.78 | ppl 17710.72
| epoka   1 |   400/ 8042 paczek | lr 5.00 | ms/paczkę 3090.53 | strata  9.04 | ppl  8402.46
| epoka   1 |   400/ 8042 paczek | lr 5.00 | ms/paczkę 3090.53 | strata  9.04 | ppl  8402.46
| epoka   1 |   600/ 8042 paczek | lr 5.00 | ms/paczkę 3045.25 | strata  8.67 | ppl  5807.50
| epoka   1 |   600/ 8042 paczek | lr 5.00 | ms/paczkę 3045.25 | strata  8.67 | ppl  5807.50
| epoka   1 |   800/ 8042 paczek | lr 5.00 | ms/paczkę 3026.61 | strata  8.48 | ppl  4838.25
| epoka   1 |   800/ 8042 paczek | lr 5.00 | ms/paczkę 3026.61 | strata  8.48 | ppl  4838.25
| epoka   1 |  1000/ 8042 paczek | lr 5.00 | ms/paczkę 2951.66 | strata  8.32 | ppl  4114.94
| epoka   1 |  1000/ 8042 paczek | lr 5.00 | ms/paczkę 2951.66 | strata  8.32 | ppl  4114.94
| epoka   1 |  1200/ 8042

In [ ]:

# Krok 9: Trening modelu Transformer (z limitem czasowym)

# Hiperparametry
ntokens = len(vocab)
emsize = 200  
nhead = 2     
nhid = 200    
nlayers = 2   
dropout = 0.2
lr = 5.0
epochs = 1 
max_training_time_hours = 4 
max_training_time_seconds = max_training_time_hours * 3600

model_transformer = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_transformer = torch.optim.SGD(model_transformer.parameters(), lr=lr)
scheduler_transformer = torch.optim.lr_scheduler.StepLR(optimizer_transformer, 1.0, gamma=0.95)

best_val_loss = float('inf')
total_training_start_time = time.time()

print(f"=== Rozpoczynam trening Transformera (limit: {max_training_time_hours}h) ===")
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model_transformer, train_data, optimizer_transformer, scheduler_transformer, criterion, epoch)
    val_loss = evaluate(model_transformer, val_data, criterion)
    
    elapsed_total_time = time.time() - total_training_start_time
    
    print('-' * 89)
    print(f'| koniec epoki {epoch:3d} | czas: {(time.time() - epoch_start_time):5.2f}s | '
          f'strata walidacyjna {val_loss:5.2f} | ppl walidacyjny {math.exp(val_loss):8.2f}')
    print(f'| Całkowity czas treningu: {elapsed_total_time/3600:.2f}h / {max_training_time_hours}h')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_transformer.state_dict(), 'best_model_transformer.pt')
        print("Zapisano nowy najlepszy model Transformer.")

    scheduler_transformer.step()
    
    if elapsed_total_time >= max_training_time_seconds:
        print(f"Przekroczono maksymalny czas treningu. Zakończono.")
        break


c:\Users\lchec\miniconda3\envs\ling-lab\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


=== Rozpoczynam trening Transformera (limit: 4h) ===
| epoka   1 |   200/ 8042 paczek | lr 5.00 | ms/paczkę 3034.99 | strata 10.68 | ppl 43599.39
| epoka   1 |   200/ 8042 paczek | lr 5.00 | ms/paczkę 3034.99 | strata 10.68 | ppl 43599.39
| epoka   1 |   400/ 8042 paczek | lr 5.00 | ms/paczkę 2994.77 | strata  9.26 | ppl 10523.77
| epoka   1 |   400/ 8042 paczek | lr 5.00 | ms/paczkę 2994.77 | strata  9.26 | ppl 10523.77
| epoka   1 |   600/ 8042 paczek | lr 5.00 | ms/paczkę 3016.38 | strata  8.71 | ppl  6048.75
| epoka   1 |   600/ 8042 paczek | lr 5.00 | ms/paczkę 3016.38 | strata  8.71 | ppl  6048.75
| epoka   1 |   800/ 8042 paczek | lr 5.00 | ms/paczkę 3061.92 | strata  8.49 | ppl  4848.09
| epoka   1 |   800/ 8042 paczek | lr 5.00 | ms/paczkę 3061.92 | strata  8.49 | ppl  4848.09
| epoka   1 |  1000/ 8042 paczek | lr 5.00 | ms/paczkę 3019.88 | strata  8.35 | ppl  4232.37
| epoka   1 |  1000/ 8042 paczek | lr 5.00 | ms/paczkę 3019.88 | strata  8.35 | ppl  4232.37
| epoka   1 |  12

In [ ]:
# Krok 10: Ewaluacja najlepszych modeli i wyniki promtowania

import torch.nn.functional as F
import os

# Krok 10.1: Funkcja do generowania tekstu
def generate_text(model, start_text, num_words, temperature=1.0):
    """
    Generuje tekst na podstawie wytrenowanego modelu.
    """
    model.eval()
    words = tokenizer(start_text)
    
    hidden = model.init_hidden(1) if isinstance(model, LSTMModel) else None
    
    src_mask = None

    with torch.no_grad():
        for word in words:
            input_tensor = torch.tensor([vocab[word]], dtype=torch.long).view(1, 1).to(device)
            if isinstance(model, LSTMModel):
                output, hidden = model(input_tensor, hidden)
            else: # Transformer
                if src_mask is None or src_mask.size(0) != len(input_tensor):
                    src_mask = model.generate_square_subsequent_mask(len(input_tensor)).to(device)
                output = model(input_tensor, src_mask)

        for _ in range(num_words):
            word_weights = output.squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            
            input_tensor = torch.tensor([word_idx], dtype=torch.long).view(1, 1).to(device)

            if isinstance(model, LSTMModel):
                output, hidden = model(input_tensor, hidden)
            else: 
                if src_mask is None or src_mask.size(0) != len(input_tensor):
                    src_mask = model.generate_square_subsequent_mask(len(input_tensor)).to(device)
                output = model(input_tensor, src_mask)

            word = vocab.lookup_token(word_idx)
            words.append(word)

    return ' '.join(words)

prompts = [
    "Dawno, dawno temu za siedmioma górami",
    "W starym zamku na wzgórzu",
    "Pewnego dnia, gdy słońce chyliło się ku zachodowi",
    "Technologia przyszłości zmieni nasze życie",
    "Najważniejszą rzeczą w życiu jest",
    "Smok otworzył swoją paszczę i",
    "Statek kosmiczny wylądował na nieznanej planecie",
    "W głębi mrocznego lasu",
    "Recepta na szczęście jest prosta",
    "Ostatni człowiek na Ziemi usłyszał pukanie do drzwi"
]

results_dir = "results"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

models_to_test = {
    "lstm": ("best_model_lstm.pt", LSTMModel(ntokens, emsize, nhid, nlayers, dropout).to(device)),
    "transformer": ("best_model_transformer.pt", TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device))
}

for model_name, (model_path, model_instance) in models_to_test.items():
    print(f"\n--- Testowanie modelu: {model_name.upper()} ---")
    if not os.path.exists(model_path):
        print(f"Nie znaleziono pliku z modelem: {model_path}. Pomijam.")
        continue
        
    model_instance.load_state_dict(torch.load(model_path))
    
    for i, prompt in enumerate(prompts):
        print(f"Generowanie dla promptu {i+1}/{len(prompts)}...")
        generated_text = generate_text(model_instance, start_text=prompt, num_words=50, temperature=0.8)
        
        output_filename = os.path.join(results_dir, f"{model_name}_prompt_{i+1}.txt")
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write("--- PROMPT ---\n")
            f.write(prompt + "\n\n")
            f.write("--- WYGENEROWANY TEKST ---\n")
            f.write(generated_text)
            
print("\nZakończono generowanie. Wyniki znajdują się w folderze 'results'.")

NameError: name 'LSTMModel' is not defined